In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom

In [2]:
expname = 'homoRB_DAS'
outpath = '../../FDModelling/homoRB_DAS/'

# Rock Properties - subsurface models

In [3]:
abs_bounds = 30 # boundary points

dx = dy = dz = 0.005  # 5mm
dimlims = 0.4 + 2*(abs_bounds*dx) # 40cm cubed
nx = ny = nz = int(dimlims/dx) 
print(nx)
# dimlims = nx*dx

# Subsurface properties
vp = 5260  # m/s
vs = 3260  # m/s
rho = 2590;  # kg/m3


140


In [4]:
subsurface_properties = [vp, vs, rho]
vp_mod, vs_mod, rho_mod = make_homo_model(dimlims,
                                            dx,
                                            subsurface_properties,
                                            pad=False,
                                            npad=0,
                                            freesurface=False
                                            )

# Write as binary
os.makedirs(os.path.join(outpath,'inputs/model/'), exist_ok=True)
vp_mod.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.vp'%expname))#,format='<f')
vs_mod.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.vs'%expname))  #,format='<f')
rho_mod.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.rho'%expname))  #,format='<f')

# Acquisition Geometry - Receivers & Source

In [5]:
# Source # Source 
sx, sy, sz = 0.2, 0.2, 0.2
sx = sy =sz = 0.2 + (abs_bounds*dx)
td = 1e-6  # ignition time
fc = 40000  # central frequency
amp = 1.  # amplitude
string_list = [str(i) for i in [sx, sz, sy, td, fc, amp]]
src_string = '\t'.join(string_list)
print(src_string)

text_file = open(os.path.join(outpath,'inputs/sources.dat'), "wt")
text_file.writelines(src_string)
text_file.close()

0.35	0.35	0.35	1e-06	40000	1.0


In [6]:
# RECEIVERS
recbuf_gps = 1
nrperface = nx - (2*abs_bounds) - (2*recbuf_gps)

# don't need to change below
nxyz = [nx-2*abs_bounds, ny-2*abs_bounds, nz-2*abs_bounds]
dxyz = [dx, dy, dz]
gridded_recs, nr, rec_string = recgeom(nxyz, dxyz, nrperfc_xyz=[nrperface], recbuf_gps=recbuf_gps)
print (nr, gridded_recs.shape)
# print(gridded_recs)
gridded_recs = gridded_recs + (abs_bounds*dx)
print(gridded_recs)

np.savetxt(os.path.join(outpath,'inputs/receiver.dat'),
           gridded_recs.T.astype(np.float32), 
           fmt='%.4f')

36504 (3, 36504)
[[0.155 0.155 0.155 ... 0.535 0.54  0.545]
 [0.16  0.16  0.16  ... 0.545 0.545 0.545]
 [0.16  0.165 0.17  ... 0.55  0.55  0.55 ]]


# BUILD JSON FOR SOFI

In [7]:
# mod params
n_xzy = vp_mod.transpose(0,2,1).shape
d_xzy = [dx, dz, dy]

# time params (in seconds!)
dt = 1e-7
tdur = 0.0005
tsnap_params = [dt, dt*4, dt*2]
smgrm_ndt = 1

SOFIjsonfilename = os.path.join(outpath,'SOFI3D_%s.json'%expname)

sofimaster = '~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/'
num_procs = 8
sofi_param_file = "SOFI3D_%s.json"%expname
shellfilename = os.path.join(outpath,'runSOFI3D_%s.sh'%expname)


In [8]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings()

monitor_strs = get_monitor_str(tsnap_params, smgrm_ndt, expname, sbsmp_xyz=2)
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xzy, d_xzy,expname)
bndry_str = get_boundary_str(fs=False, npad=abs_bounds, cpml=False)
src_str = get_source_str()

write_SOFIjsonParams = write_SOFIjsonParams(default_strs, 
                                            monitor_strs,  
                                            t_str, 
                                            mod_strs, 
                                            src_str, 
                                            bndry_str, 
                                            SOFIjsonfilename)
runscript = write_SOFIrunScript(sofimaster, 
                                num_procs, 
                                sofi_param_file, 
                                shellfilename)

In [9]:
vp_mod.transpose(0,2,1).shape

(140, 140, 140)